## Import packages

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import cline_analysis as ca
import pandas as pd
import seaborn as sns
import datetime
import os
from scipy.signal import medfilt
import functools
from scipy.optimize import bisect
from scipy import stats

sns.set_style("whitegrid")
sns.set_style("ticks")
%matplotlib qt
%config InlineBackend.figure_format = 'svg'
plt.matplotlib.rcParams['svg.fonttype'] = 'svgfont' # fonts will be recognized by Adobe Illustrator

## Load data

In [5]:
dirname = '/Users/zoltan/Dropbox/Channels/Fluvial/Yavari/csv_files/'
fnames,clxs,clys,rbxs,lbxs,rbys,lbys,curvatures,ages,widths,dates = ca.load_data(dirname)

In [6]:
fnames

['Yavari_19880715.csv',
 'Yavari_20160813.csv',
 'Yavari_tributary_1_19880715.csv',
 'Yavari_tributary_1_20160813.csv']

In [7]:
dates

[datetime.datetime(1988, 7, 15, 0, 0),
 datetime.datetime(2016, 8, 13, 0, 0),
 datetime.datetime(1988, 7, 15, 0, 0),
 datetime.datetime(2016, 8, 13, 0, 0)]

## Get migration rate

In [8]:
ts1 = 2 # first timestep
ts2 = 3 # second timestep

d = dates[ts2]-dates[ts1]
years = d.days/365.0

x = np.array(clxs[ts1])
y = np.array(clys[ts1])

xn = np.array(clxs[ts2])
yn = np.array(clys[ts2])

migr_rate, migr_sign, p, q = ca.get_migr_rate(x,y,xn,yn,years,0)

In [8]:
migr_rate = medfilt(savgol_filter(migr_rate,41,3),kernel_size=5) # smoothing
curv,s = ca.compute_curvature(x,y)
curv = medfilt(savgol_filter(curv,71,3),kernel_size=5) # smoothing

In [9]:
# set intervals affected by cu=toffs to NaN - specific to Yavari tributary river segment
migr_rate[1694:1744] = np.NaN
migr_rate[2444:2665] = np.NaN
migr_rate[3442:3595] = np.NaN

In [10]:
plt.figure()
plt.plot(migr_rate)

## Read 'valid' inflection points and corresponding points of zero migration from CSV file

In [11]:
df = pd.read_csv('Yavari_LT05_L1TP_005063_19880715_20170208_01_T1_tributary_1_inflection_and_zero_migration_indices.csv')
LZC = np.array(df['index of inflection point'])
LZM = np.array(df['index of zero migration'])

In [12]:
# indices of bends affected by low erodibility and cutoffs (these have been picked manually)
erodibility_inds = [66,81,83,85,98,105,118,120,124,130]
cutoff_inds = [26,27,36,45,48,49,93,94,122,123]

## Plot curvature and migration rate series side-by-side

In [29]:
# plot curvature and migration rate along the channel

W = np.nanmean(widths[2])

fig, ax1 = plt.subplots(figsize=(25,4))
plt.tight_layout()

curv_scale = 0.4
migr_scale = 2
y1 = curv_scale
y2 = -3*curv_scale
y3 = 3*migr_scale
y4 = -migr_scale

y5 = -2*curv_scale
y6 = 2*migr_scale

for i in range(0,len(LZC)-1,2):
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,0.85,0.85],edgecolor='k',zorder=0)

deltas = 25.0
ax1.fill_between(s, 0, curv*W)
ax2 = ax1.twinx()
ax2.fill_between(s, 0, migr_rate, facecolor='green')

ax1.plot([0,max(s)],[0,0],'k--')
ax2.plot([0,max(s)],[0,0],'k--')

ax1.set_ylim(y2,y1)
ax2.set_ylim(y4,y3)
ax1.set_xlim(0,s[-1])

for i in erodibility_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[1.0,0.85,0.85],edgecolor='k',zorder=0) 
    
for i in cutoff_inds:
    xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
    ycoords = [y1,y1,0,y5,y2,y2,y5,0]
    ax1.fill(xcoords,ycoords,facecolor=[0.85,1.0,0.85],edgecolor='k',zorder=0) 
    
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y1,y1,0,y5,y2,y2,y5,0]
        ax1.fill(xcoords,ycoords,color='w') 
        
for i in range(len(LZC)-1):
    if np.sum(np.isnan(migr_rate[LZM[i]:LZM[i+1]]))>0:
        xcoords = [s[LZC[i]],s[LZC[i+1]],s[LZC[i+1]],s[LZM[i+1]],s[LZM[i+1]],s[LZM[i]],s[LZM[i]],s[LZC[i]]]
        ycoords = [y3,y3,y6,0,y4,y4,0,y6]
        ax2.fill(xcoords,ycoords,color='w') 

for i in range(0,len(LZC)-1,2):
    ax1.text(s[LZC[i]],0.4,str(i),fontsize=12)


## Estimate lag between curvature and migration rate

In [17]:
window_length = 500
time_shifts = ca.get_time_shifts(migr_rate,curv,window_length)

In [18]:
plt.figure()
plt.plot(time_shifts)

In [19]:
# average lag
25.0*np.round(np.mean(time_shifts))

-450.0

In [20]:
# average lag estimated from distances between inflection points and points of zero migration 
# (this is what was used in the paper)
np.mean(25.0*(LZM-LZC))

406.98529411764707

## Estimate friction factor Cf

In [30]:
migr_rate_t = migr_rate.copy()
migr_rate_t[1694:1744] = 0
migr_rate_t[2444:2665] = 0
migr_rate_t[3442:3595] = 0

curv_t = curv.copy()
curv_t[1694:1744] = 0
curv_t[2444:2665] = 0
curv_t[3442:3595] = 0

In [31]:
# this might take a while to run
kl = 3.0 # preliminary kl value (guesstimate)
k = 1
D = (W/18.8)**0.7092 # depth in meters (from width)

# dx,dy,ds,s = ca.compute_derivatives(xt,yt)
# curv_t, s = ca.compute_curvature(xt,yt)
# curv_t = medfilt(savgol_filter(curv_t,71,3),kernel_size=5) # smoothing

# migr_rate_t = medfilt(savgol_filter(migr_rate_t,71,3),kernel_size=5)

get_friction_factor_1 = functools.partial(ca.get_friction_factor,curvature=curv_t,migr_rate=migr_rate_t,
                                          kl=kl,W=W, k=k, D=D, s=s)

Cf_opt = bisect(get_friction_factor_1, 0.0002, 0.1)
print Cf_opt

0.00526796875


In [32]:
Cf_opt = 0.00526796875

## Estimate migration rate constant kl

In [33]:
# minimize the error between actual and predicted migration rates (using the 75th percentile)
errors = []
# curv_t, s = ca.compute_curvature(xt,yt)
for i in np.arange(1,10):
    print i
    R1 = ca.get_predicted_migr_rate(curv_t,W=W,k=1,Cf=Cf_opt,D=D,kl=i,s=s)
    errors.append(np.abs(np.percentile(np.abs(R1),75)-np.percentile(np.abs(migr_rate_t[1:-1]),75)))
    
plt.figure()
plt.plot(np.arange(1,10),errors);

1
2
3
4
5
6
7
8
9


In [34]:
kl_opt = 4.0 # the error is at minimum for kl = 4.0

In [28]:
407/25.0

16.28

## Plot actual migration rate against nominal migration rate

In [43]:
# kernel density and scatterplot of actual vs. nominal migration rate
w = np.nanmedian(widths[2]) 
curv_nodim = w*curv*kl_opt
lag = 16
plt.figure(figsize=(8,8))
sns.kdeplot(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0], migr_rate[lag:][np.isnan(migr_rate[lag:])==0],
           n_levels=20,shade=True,cmap='Blues',shade_lowest=False)
plt.scatter(curv_nodim[:-lag][::20],migr_rate[lag:][::20],c='k',s=15)
max_x = 2.5
plt.xlim(-max_x,max_x)
plt.ylim(-max_x,max_x)
plt.plot([-max_x,max_x],[-max_x,max_x],'k--')
plt.xlabel('nominal migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [44]:
# get correlation coefficient for relationship between curvature and migration rate
slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0],
                                                                      migr_rate[lag:][np.isnan(migr_rate[lag:])==0])
print r_value
print r_value**2
print p_value

0.784497877311
0.615436919505
0.0


In [37]:
# number of data points used in analysis
len(curv_nodim[:-lag][np.isnan(migr_rate[lag:])==0])

8668

In [38]:
# compute predicted migration rates
D = (w/18.8)**0.7092 # depth in meters (from width)
dx,dy,ds,s = ca.compute_derivatives(x,y)
R1 = ca.get_predicted_migr_rate(curv,W=w,k=1,Cf=Cf_opt,D=D,kl=kl_opt,s=s)

In [39]:
# plot actual and predicted migration rates
plt.figure()
plt.plot(s,migr_rate)
plt.plot(s,R1,'r')

In [40]:
# get correlation coefficient for relationship between actual and predicted migration rate
m_nonan = migr_rate[(np.isnan(R1)==0)&(np.isnan(migr_rate)==0)]
R_nonan = R1[(np.isnan(R1)==0)&(np.isnan(migr_rate)==0)]

slope, intercept, r_value, p_value, slope_std_rror = stats.linregress(R_nonan,m_nonan)
print r_value
print r_value**2
print p_value

0.773642177706
0.598522219126
0.0


In [41]:
# 90th percentile of migration rate
np.percentile(np.abs(m_nonan),90)

1.7905648228134488

In [45]:
# plot actual vs. predicted migration rate
max_m = 2.5
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

In [48]:
# plot actual vs. predicted migration rate
max_m = 4.0
plt.figure(figsize=(8,8))
sns.kdeplot(R_nonan,m_nonan,n_levels=10,shade=True,cmap='Blues',shade_lowest=False)
plt.plot([-max_m,max_m],[-max_m,max_m],'k--') 
plt.scatter(R_nonan[::20],m_nonan[::20],c='k',s=15)
plt.xlim(-max_m,max_m)
plt.ylim(-max_m,max_m)
plt.xlabel('predicted migration rate (m/year)', fontsize=14)
plt.ylabel('actual migration rate (m/year)', fontsize=14)

# add points affected by cutoffs and low erodibility
for i in erodibility_inds:
    plt.scatter(R1[LZC[i]:LZC[i+1]][::5],migr_rate[LZC[i]:LZC[i+1]][::5],c='r',s=15)
for i in cutoff_inds:
    plt.scatter(R1[LZC[i]:LZC[i+1]][::5],migr_rate[LZC[i]:LZC[i+1]][::5],c='g',s=15)